In [1]:
import pandas as pd
import numpy as np
import pymysql
from datetime import datetime
import warnings  
warnings.filterwarnings("ignore")

In [2]:
def query(sql,
          host="rr-wz9wx0w3yti9d4f6wro.mysql.rds.aliyuncs.com",
          user="ylc",
          password="1O8t5lcJ5aMhwwPEUUjS",
          database = '' ,
          port=3306
         ):   
    conn = pymysql.connect(
        host=host,
        user=user,
        port=port,
        password=password,
        max_allowed_packet=1073741824,
        charset="utf8")
    try:
        df = pd.read_sql(sql, con=conn)
        conn.close()
    except:
        print('error')
        conn.close()
        raise
    return df

In [3]:
sql = '''
SELECT om.order_number,om.`status`,tos.refund_date, tos.money, tos.real_pay_money, tos.pay_discount_money, toou.pay_status, toou.pay_date, toou.real_pay_money as real_pay_money_t, toou.money as money_t
FROM db_digua_business.t_order om
left join db_digua_business.t_order_out toou on toou.order_id=om.id
left join db_digua_business.t_order_stages tos on tos.order_id=om.id
WHERE om.`status` in (3,4) and om.has_order_check='Y'
'''
df = query(sql)

In [4]:
sql1 = '''
select 
tor.money, tor.relet_start_time, tor.cycle_status
from db_digua_business.t_order om 
left join db_digua_business.t_order_relet tor on tor.order_id=om.id
WHERE om.`status` in (3,4,8) and om.has_order_check='Y'
'''
df1 = query(sql1)

In [5]:
df.loc[:, 'refund_date'] = df.refund_date.dt.date
df.loc[:, 'pay_date'] = df.pay_date.dt.date
now_date = pd.to_datetime(datetime.now().strftime('%Y-%m-%d'))
now_date
# df.refund_date 
df.loc[:, '实际支付金额'] = np.where(df.refund_date<=now_date, df.real_pay_money+df.pay_discount_money, 0)
df.loc[:, '实际买断金额'] = np.where((df.pay_date<=now_date)&(df.pay_status==2), df.real_pay_money_t, 0)
df.loc[:, '预计支付金额'] = np.where(df.refund_date<=now_date, df.money, 0)

In [6]:
df1.loc[:, '实际支付续租金额'] = np.where((df1.relet_start_time<=now_date)&(df1.cycle_status==2), df1.money, 0)
df1.loc[:, '预计支付续租金额'] = np.where(df1.relet_start_time<=now_date, df1.money, 0)

In [13]:
df_new = pd.concat([df[['实际支付金额', '实际买断金额', '预计支付金额']], df1[['实际支付续租金额', '预计支付续租金额']]], axis=1)
df_new[['实际支付金额', '实际买断金额', '预计支付金额','实际支付续租金额', '预计支付续租金额']].sum()


实际支付金额      77997086.26
实际买断金额        406645.80
预计支付金额      88907839.04
实际支付续租金额     6649876.84
预计支付续租金额     8616799.47
dtype: float64

In [15]:
# 分子：到达扣款时间内的实付租金+支付时间在时间范围内的实付买断金额+达到扣款时间内的实付续租租金
# 分母：到达扣款时间内的应付租金+支付时间在时间范围内的实付买断金额+达到扣款时间内的应付续租租金
(df_new['实际支付金额'].sum()+df_new['实际买断金额'].sum()+df_new['实际支付续租金额'].sum())/(df_new['预计支付金额'].sum()+df_new['实际买断金额'].sum()+df_new['预计支付续租金额'].sum())

np.float64(0.8685029457059306)